# Install `simex`

In [22]:
%%bash
#clone repo of `sipecam-metadata-extactor` in server
#git clone git@github.com:CONABIO/sipecam-metadata-extractor.git ~/
cd ../src
#install in editable mode
pip install -q -e .

# Or `docker run`

``` 
SIMEX_VERSION=0.1
REPO_URL=sipecam/simex
CONTAINER_NAME=sipecam-simex
docker run --rm -v $HOME:/shared_volume --name $CONTAINER_NAME -p 3000:8888 -d $REPO_URL:$SIMEX_VERSION /usr/local/bin/jupyter lab --ip=0.0.0.0 --no-browser --allow-root
```

# Copy files for testing

In [1]:
%%bash
cp -r /shared_volume/testing_simex_backup/sacmod /shared_volume/testing_simex/

# List of files to extract metadata

In [2]:
%%bash
dir_with_sipecam_data=/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM
list_of_files_and_subdirectories_to_extract_metadata --input_directory $dir_with_sipecam_data

## Check

In [3]:
%%bash
ls -lh /shared_volume/sipecam_files_to_extract_metadata_from_*

-rw-r--r-- 1 root root 3.9K Mar  8 09:43 /shared_volume/sipecam_files_to_extract_metadata_from_08-03-2022.txt


In [4]:
%%bash
ls -lh /shared_volume/sipecam_subdirectories_*

-rw-r--r-- 1 root root 344 Mar  8 09:43 /shared_volume/sipecam_subdirectories_08-03-2022.txt


In [5]:
%%bash
wc -l /shared_volume/sipecam_files_to_extract_metadata_from_*

30 /shared_volume/sipecam_files_to_extract_metadata_from_08-03-2022.txt


In [6]:
%%bash
wc -l /shared_volume/sipecam_subdirectories_*

3 /shared_volume/sipecam_subdirectories_08-03-2022.txt


In [7]:
%%bash
#check every first subdirectory has a JPG, WAV or AVI file
SAVEIFS=$IFS;IFS=$(echo -en "\n\b")
for f in $(cat /shared_volume/sipecam_subdirectories_*.txt)
  do echo "$f"
     ls "$f"|grep "[WAV|wav|JPG|jpg|AVI|avi]$"|head -n 1
done
IFS=$SAVEIFS

/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico
20210728_180000.WAV
/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible
20210729_180000.WAV
/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX
RCNX0001.JPG


# Generate SiPeCaM Zendro schema (one time only)

In [ ]:
%%bash
echo "SIPECAM_ZENDRO_GQL_URL=https://gql.sipecamdata.conabio.gob.mx/" > ~/.simex_env
echo "SIPECAM_ZENDRO_GQL_USER=<user>" >> ~/.simex_env
echo "SIPECAM_ZENDRO_GQL_PASSWORD=<password>" >> ~/.simex_env


In [13]:
%%bash
generate_sipecam_zendro_schema

Standard Error sgqlc.introspection b''
Standard Output sgqlc.introspection b'\n'
Standard Error sgqlc-codegen b''
Standard Output sgqlc-codegen b''


## Check

In [8]:
%%bash
head -n 10 /root/sipecam-metadata-extractor/src/simex/sipecam_zendro_schema.py

import sgqlc.types
import sgqlc.types.datetime


sipecam_zendro_schema = sgqlc.types.Schema()



########################################################################
# Scalars and Enumerations


# Extract serial number, dates and metadata of files and device

In [8]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/"
#default for parallel execution are 4 processes
extract_serial_numbers_dates_and_metadata_of_files_and_device --input_dir "$dir_to_be_processed" --parallel

In [9]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico"
#can select number of proceses
extract_serial_numbers_dates_and_metadata_of_files_and_device --input_dir "$dir_to_be_processed" --parallel --number_of_processes 6

In [10]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible"
#if number_of_processes is passed as arg then is assumed will execute in parallel
extract_serial_numbers_dates_and_metadata_of_files_and_device --input_dir "$dir_to_be_processed" --number_of_processes 6

## Check

### Dir with image & video files

In [11]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/"
file_for_logs=$(find "$dir_to_be_processed" -name "logs_simex_extract_serial_numbers_dates_and_metadata_of_files_and_device.logs")
head -n 15 "$file_for_logs"

2022-03-08 09:43:33,643 [INFO]  extraction of serial_numbers_dates_and_metadata_of_files_and_device
2022-03-08 09:43:33,643 [INFO]  logs for extraction of serial_numbers_dates_and_metadata_of_files_and_device in /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/100RECNX_simex_extract_serial_numbers_dates_and_metadata_of_files_and_device.json
2022-03-08 09:43:33,644 [INFO]  extraction of metadata of device from /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/RCNX0074.JPG
2022-03-08 09:43:33,816 [INFO]  SUCCESSFUL extraction of serial number of /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/RCNX0074.JPG
2022-03-08 09:43:35,782 [INFO]  extraction of date of /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/RCNX0074.JPG
2022-03-0

In [12]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/"
file_json=$(find "$dir_to_be_processed" -name "*.json")
python3 -mjson.tool "$file_json"|head -n 80

{
    "MetadataDevice": {
        "Make": "RECONYX",
        "Model": "HF2 PRO COVERT",
        "SerialNumber": "HLPXGM09048890"
    },
    "MetadataFiles": {
        "/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/RCNX0074.JPG": {
            "Make": "RECONYX",
            "Model": "HF2 PRO COVERT",
            "DateTimeOriginal": "2021:08:26 13:12:35",
            "SerialNumber": "HLPXGM09048890",
            "FileSize": "627 KiB",
            "ExifByteOrder": "Little-endian (Intel, II)",
            "ImageWidth": 2048,
            "ImageHeight": 1440,
            "EncodingProcess": "Baseline DCT, Huffman coding",
            "BitsPerSample": 8,
            "ColorComponents": 3,
            "YCbCrSubSampling": "YCbCr4:2:2 (2 1)",
            "XResolution": 72,
            "YResolution": 72,
            "ResolutionUnit": "inches",
            "YCbCrPositioning": "Co-sited",
            "ExposureTime": "1/30",
        

In [13]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/"
file_json=$(find "$dir_to_be_processed" -name "*.json")
python3 -mjson.tool "$file_json"|tail -n 50

            "MaxDataRate": 26542080,
            "Megapixels": 0.73728,
            "GPSLatitudeRef": "North",
            "GPSLongitudeRef": "West",
            "GPSLatitude": 16.1583,
            "GPSLongitude": -90.9048
        },
        "/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/RCNX0075.AVI": {
            "BitRate": "11.3 Mbit/sec",
            "Endianness": "Little endian",
            "VideoBitsPerPixel": "24",
            "VideoCompression": "Motion JPEG DIB (fourcc:\"MJPG\")",
            "AudioSampleRate": "16.0 kHz",
            "AudioCompressionRate": "1.0x",
            "AudioCompression": "Microsoft Pulse Code Modulation (PCM)",
            "AudioBitRate": "256.4 Kbit/sec",
            "FileSize": "6.0 MiB",
            "DateTimeOriginal": "2021:08:26 13:12:35",
            "BMPVersion": "Windows V3",
            "ImageWidth": 1024,
            "ImageHeight": 720,
            "Planes": 1,
         

### Dir with audio files

In [14]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico"
file_for_logs=$(find "$dir_to_be_processed" -name "logs_simex_extract_serial_numbers_dates_and_metadata_of_files_and_device.logs")
head -n 15 "$file_for_logs"

2022-03-08 09:43:36,106 [INFO]  extraction of serial_numbers_dates_and_metadata_of_files_and_device
2022-03-08 09:43:36,107 [INFO]  logs for extraction of serial_numbers_dates_and_metadata_of_files_and_device in /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico/Ultrasonico_simex_extract_serial_numbers_dates_and_metadata_of_files_and_device.json
2022-03-08 09:43:36,107 [INFO]  extraction of metadata of device from /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico/20210728_184000.WAV
2022-03-08 09:43:36,469 [INFO]  SUCCESSFUL extraction of serial number of /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico/20210728_184000.WAV
2022-03-08 09:43:36,470 [INFO]  GPSFile key not found for videos, files from directory are of audio
2022-03-08 09:43:38,070 [INFO]  extraction of date of /shared_volume/te

In [15]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico"
file_json=$(find "$dir_to_be_processed" -name "*.json")
python3 -mjson.tool "$file_json" | head -n 40

{
    "MetadataDevice": {
        "Artist": "AudioMoth 2423C2045F256D74",
        "SerialNumber": "2423C2045F256D74"
    },
    "MetadataFiles": {
        "/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico/20210728_184000.WAV": {
            "Battery": "4.8V",
            "Datetime": "18:40:00 28/07/2021 (UTC-0600)",
            "Gain": "medium",
            "Timezone": "UTC",
            "BitRate": "6.1 Mbit/sec",
            "SerialNumber": "2423C2045F256D74",
            "FileSize": "22 MiB",
            "Encoding": "Microsoft PCM",
            "NumChannels": 1,
            "SampleRate": 384000,
            "AvgBytesPerSec": 768000,
            "BitsPerSample": 16,
            "Comment": "Recorded at 18:40:00 28/07/2021 (UTC-6) by AudioMoth 2423C2045F256D74 at medium gain setting while battery state was 4.8V and temperature was 23.4C.",
            "Duration": "0:00:30"
        },
        "/shared_volume/testing_simex

In [16]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico"
file_json=$(find "$dir_to_be_processed" -name "*.json")
python3 -mjson.tool "$file_json" | tail -n 30

            "NumChannels": 1,
            "SampleRate": 384000,
            "AvgBytesPerSec": 768000,
            "BitsPerSample": 16,
            "Comment": "Recorded at 18:20:00 28/07/2021 (UTC-6) by AudioMoth 2423C2045F256D74 at medium gain setting while battery state was 4.8V and temperature was 24.4C.",
            "Duration": "0:00:30"
        },
        "/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico/20210826_050000.WAV": {
            "Battery": "4.4V",
            "Datetime": "05:00:00 26/08/2021 (UTC-0600)",
            "Gain": "medium",
            "Timezone": "UTC",
            "BitRate": "6.1 Mbit/sec",
            "SerialNumber": "2423C2045F256D74",
            "FileSize": "22 MiB",
            "Encoding": "Microsoft PCM",
            "NumChannels": 1,
            "SampleRate": 384000,
            "AvgBytesPerSec": 768000,
            "BitsPerSample": 16,
            "Comment": "Recorded at 05:00:00 26/0

### Dir with audio files

In [17]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible"
file_for_logs=$(find "$dir_to_be_processed" -name "logs_simex_extract_serial_numbers_dates_and_metadata_of_files_and_device.logs")
head -n 15 "$file_for_logs"

2022-03-08 09:43:38,385 [INFO]  extraction of serial_numbers_dates_and_metadata_of_files_and_device
2022-03-08 09:43:38,386 [INFO]  logs for extraction of serial_numbers_dates_and_metadata_of_files_and_device in /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible/Audible_simex_extract_serial_numbers_dates_and_metadata_of_files_and_device.json
2022-03-08 09:43:38,386 [INFO]  extraction of metadata of device from /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible/20210828_052000.WAV
2022-03-08 09:43:38,715 [INFO]  SUCCESSFUL extraction of serial number of /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible/20210828_052000.WAV
2022-03-08 09:43:38,715 [INFO]  GPSFile key not found for videos, files from directory are of audio
2022-03-08 09:43:40,309 [INFO]  extraction of date of /shared_volume/testing_simex/s

In [18]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible"
file_json=$(find "$dir_to_be_processed" -name '*.json')
python3 -mjson.tool "$file_json" | head -n 40

{
    "MetadataDevice": {
        "Artist": "AudioMoth 248D9B045EC9E6B1",
        "SerialNumber": "248D9B045EC9E6B1"
    },
    "MetadataFiles": {
        "/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible/20210828_052000.WAV": {
            "Battery": "4.4V",
            "Datetime": "05:20:00 28/08/2021 (UTC-0600)",
            "Gain": "medium",
            "Timezone": "UTC",
            "BitRate": "6.1 Mbit/sec",
            "SerialNumber": "248D9B045EC9E6B1",
            "FileSize": "22 MiB",
            "Encoding": "Microsoft PCM",
            "NumChannels": 1,
            "SampleRate": 384000,
            "AvgBytesPerSec": 768000,
            "BitsPerSample": 16,
            "Comment": "Recorded at 05:20:00 28/08/2021 (UTC-6) by AudioMoth 248D9B045EC9E6B1 at medium gain setting while battery state was 4.4V and temperature was 22.6C.",
            "Duration": "0:00:30"
        },
        "/shared_volume/testing_simex/sa

In [19]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible"
file_json=$(find "$dir_to_be_processed" -name '*.json')
python3 -mjson.tool "$file_json" | tail -n 30

            "NumChannels": 1,
            "SampleRate": 384000,
            "AvgBytesPerSec": 768000,
            "BitsPerSample": 16,
            "Comment": "Recorded at 04:20:00 28/08/2021 (UTC-6) by AudioMoth 248D9B045EC9E6B1 at medium gain setting while battery state was 4.4V and temperature was 22.9C.",
            "Duration": "0:00:30"
        },
        "/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible/20210729_190000.WAV": {
            "Battery": "4.8V",
            "Datetime": "19:00:00 29/07/2021 (UTC-0600)",
            "Gain": "medium",
            "Timezone": "UTC",
            "BitRate": "6.1 Mbit/sec",
            "SerialNumber": "248D9B045EC9E6B1",
            "FileSize": "22 MiB",
            "Encoding": "Microsoft PCM",
            "NumChannels": 1,
            "SampleRate": 384000,
            "AvgBytesPerSec": 768000,
            "BitsPerSample": 16,
            "Comment": "Recorded at 19:00:00 29/07/2

# Move files to standard directory

In [20]:
%%bash
echo "SIPECAM_ZENDRO_GQL_URL=https://gql.sipecamdata.conabio.gob.mx/" > ~/.simex_env
echo "SIPECAM_ZENDRO_GQL_USER=<user>" >> ~/.simex_env
echo "SIPECAM_ZENDRO_GQL_PASSWORD=<password>" >> ~/.simex_env


## Dir with images & videos

In [22]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/"
path_std_dir="/shared_volume/testing_simex/sipecam_simex_std_dir"
move_files_to_standard_directory --directory_with_file_of_serial_number_and_dates "$dir_to_be_processed" --path_for_standard_directory "$path_std_dir"

## Check it

In [23]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/"
file_for_logs=$(find "$dir_to_be_processed" -name 'logs_simex_move_files_to_standard_directory.logs')
head -n 75 "$file_for_logs"

2022-03-08 09:45:17,055 [INFO]  Dir /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/ has serial number HLPXGM09048890
2022-03-08 09:45:17,055 [INFO]  File /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/RCNX0005.JPG has date 2021-07-28
2022-03-08 09:45:17,055 [INFO]  File /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Camaras/1338_1/100RECNX/RCNX0075.AVI has date 2021-08-26
2022-03-08 09:45:17,055 [INFO]  DaysBetweenFirstAndLastDate: 29
2022-03-08 09:45:17,214 [INFO]  Query to Zendro GQL: query {
  physical_devices(pagination: {limit: 0}, search: {field: serial_number, value: "HLPXGM09048890", operator: like}) {
    device_deploymentsFilter(pagination: {limit: 0}, search: {operator: and, search: [{field: date_deployment, value: "2021-07-28", valueType: String, operator: gte}, {field: date_deployment, value: "20

In [24]:
%%bash
path_std_dir="/shared_volume/testing_simex/sipecam_simex_std_dir"
dir_images=$(find "$path_std_dir" -name "images_videos")
file=$(find "$dir_images" -name "*.json")
python3 -mjson.tool "$file"|head -n 60

{
    "DaysBetweenFirstAndLastDate": 29,
    "MetadataDevice": {
        "Make": "RECONYX",
        "Model": "HF2 PRO COVERT",
        "SerialNumber": "HLPXGM09048890",
        "NomenclatureNode": "3_92_1_1338",
        "CumulusName": "92",
        "DateDeployment": "2021-07-28",
        "EcosystemsName": "Selvas humedas",
        "Latitude": 16.15832,
        "Longitude": -90.90487,
        "NodeCategoryIntegrity": "Integro"
    },
    "MetadataFiles": {
        "/shared_volume/testing_simex/sipecam_simex_std_dir/92/3_92_1_1338/HLPXGM09048890/2021-07-28/images_videos/b17f1d78318f84bd411c514e56ae7f62_0074.JPG": {
            "Make": "RECONYX",
            "Model": "HF2 PRO COVERT",
            "DateTimeOriginal": "2021:08:26 13:12:35",
            "SerialNumber": "HLPXGM09048890",
            "FileSize": "627 KiB",
            "ExifByteOrder": "Little-endian (Intel, II)",
            "ImageWidth": 2048,
            "ImageHeight": 1440,
            "EncodingProcess": "Baseline DCT, Huff

In [25]:
%%bash
path_std_dir="/shared_volume/testing_simex/sipecam_simex_std_dir"
dir_images=$(find "$path_std_dir" -name "images_videos")
file=$(find "$dir_images" -name "*.json")
python3 -mjson.tool "$file"|tail -n 130

            "TimeZoneOffset": -6,
            "ComponentsConfiguration": "Y, Cb, Cr, -",
            "ColorSpace": "sRGB",
            "GPSLatitudeRef": "North",
            "GPSLongitudeRef": "West",
            "AmbientTemperature": "24 C",
            "Contrast": 128,
            "Brightness": 0,
            "Sharpness": 64,
            "Saturation": 128,
            "Flash": "Off",
            "AmbientInfrared": 120,
            "AmbientLight": 228,
            "MotionSensitivity": 80,
            "BatteryVoltage": 7429,
            "BatteryVoltageAvg": 7429,
            "GPSLatitude": 16.1583,
            "GPSLongitude": -90.9048,
            "Megapixels": 2.94912
        },
        "/shared_volume/testing_simex/sipecam_simex_std_dir/92/3_92_1_1338/HLPXGM09048890/2021-07-28/images_videos/a0b803745d9d603d52f4882d594b54af_0072.AVI": {
            "BitRate": "11.9 Mbit/sec",
            "Endianness": "Little endian",
            "VideoBitsPerPixel": "24",
            "VideoCompressio

## Dir with audio files

In [26]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico"
path_std_dir="/shared_volume/testing_simex/sipecam_simex_std_dir"
move_files_to_standard_directory --directory_with_file_of_serial_number_and_dates "$dir_to_be_processed" --path_for_standard_directory "$path_std_dir"

## Check it

In [27]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico"
file_for_logs=$(find "$dir_to_be_processed" -name 'logs_simex_move_files_to_standard_directory.logs')
head -n 100 "$file_for_logs"

2022-03-08 09:46:01,471 [INFO]  Dir /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico has serial number 2423C2045F256D74
2022-03-08 09:46:01,471 [INFO]  File /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico/20210728_184000.WAV has date 2021-07-28
2022-03-08 09:46:01,471 [INFO]  File /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon_1338/Audio/1338_1/Ultrasonico/20210826_050000.WAV has date 2021-08-26
2022-03-08 09:46:01,471 [INFO]  DaysBetweenFirstAndLastDate: 29
2022-03-08 09:46:01,632 [INFO]  Query to Zendro GQL: query {
  physical_devices(pagination: {limit: 0}, search: {field: comments, value: "ADM2423C2045F256D74", operator: like}) {
    device_deploymentsFilter(pagination: {limit: 0}, search: {operator: and, search: [{field: date_deployment, value: "2021-07-28", valueType: String, operator: gte}, {field: date_depl

In [28]:
%%bash
path_std_dir="/shared_volume/testing_simex/sipecam_simex_std_dir"
dir_audios=$(find "$path_std_dir" -name "2423C2045F256D74")
file=$(find "$dir_audios" -name "*.json")
python3 -mjson.tool "$file"|head -n 60

{
    "DaysBetweenFirstAndLastDate": 29,
    "MetadataDevice": {
        "Artist": "AudioMoth 2423C2045F256D74",
        "SerialNumber": "2423C2045F256D74",
        "NomenclatureNode": "3_92_1_1338",
        "CumulusName": "92",
        "DateDeployment": "2021-07-28",
        "EcosystemsName": "Selvas humedas",
        "Latitude": 16.15832,
        "Longitude": -90.90487,
        "NodeCategoryIntegrity": "Integro"
    },
    "MetadataFiles": {
        "/shared_volume/testing_simex/sipecam_simex_std_dir/92/3_92_1_1338/2423C2045F256D74/2021-07-28/audios/Ultrasonico/6347b1d99aa5099ed4e6d811fda5052b.WAV": {
            "Battery": "4.8V",
            "Datetime": "18:40:00 28/07/2021 (UTC-0600)",
            "Gain": "medium",
            "Timezone": "UTC",
            "BitRate": "6.1 Mbit/sec",
            "SerialNumber": "2423C2045F256D74",
            "FileSize": "22 MiB",
            "Encoding": "Microsoft PCM",
            "NumChannels": 1,
            "SampleRate": 384000,
            "

## Dir with audio files

In [29]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible"
path_std_dir="/shared_volume/testing_simex/sipecam_simex_std_dir"
move_files_to_standard_directory --directory_with_file_of_serial_number_and_dates "$dir_to_be_processed" --path_for_standard_directory "$path_std_dir"

## Check it

In [30]:
%%bash
dir_to_be_processed="/shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible"
file_for_logs=$(find "$dir_to_be_processed" -name 'logs_simex_move_files_to_standard_directory.logs')
head -n 100 "$file_for_logs"

2022-03-08 09:46:41,316 [INFO]  Dir /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible has serial number 248D9B045EC9E6B1
2022-03-08 09:46:41,316 [INFO]  File /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible/20210729_184000.WAV has date 2021-07-29
2022-03-08 09:46:41,316 [INFO]  File /shared_volume/testing_simex/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia_1339/1339_2/Audio/Audible/20210828_042000.WAV has date 2021-08-28
2022-03-08 09:46:41,316 [INFO]  DaysBetweenFirstAndLastDate: 30
2022-03-08 09:46:41,475 [INFO]  Query to Zendro GQL: query {
  physical_devices(pagination: {limit: 0}, search: {field: comments, value: "ADM248D9B045EC9E6B1", operator: like}) {
    device_deploymentsFilter(pagination: {limit: 0}, search: {operator: and, search: [{field: date_deployment, value: "2021-07-29", valueType: String, operator: gte}, {field: date_deployment, v

In [31]:
%%bash
path_std_dir="/shared_volume/testing_simex/sipecam_simex_std_dir"
dir_audios=$(find "$path_std_dir" -name "248D9B045EC9E6B1")
file=$(find "$dir_audios" -name "*.json")
python3 -mjson.tool "$file"|head -n 60

{
    "DaysBetweenFirstAndLastDate": 30,
    "MetadataDevice": {
        "Artist": "AudioMoth 248D9B045EC9E6B1",
        "SerialNumber": "248D9B045EC9E6B1",
        "NomenclatureNode": "3_92_0_1339",
        "CumulusName": "92",
        "DateDeployment": "2021-07-29",
        "EcosystemsName": "Selvas humedas",
        "Latitude": 16.1517,
        "Longitude": -90.81243,
        "NodeCategoryIntegrity": "Degradado"
    },
    "MetadataFiles": {
        "/shared_volume/testing_simex/sipecam_simex_std_dir/92/3_92_0_1339/248D9B045EC9E6B1/2021-07-29/audios/Ultrasonico/41c38ee40d87bedcab44487b52ceb8b6.WAV": {
            "Battery": "4.4V",
            "Datetime": "05:20:00 28/08/2021 (UTC-0600)",
            "Gain": "medium",
            "Timezone": "UTC",
            "BitRate": "6.1 Mbit/sec",
            "SerialNumber": "248D9B045EC9E6B1",
            "FileSize": "22 MiB",
            "Encoding": "Microsoft PCM",
            "NumChannels": 1,
            "SampleRate": 384000,
            

# Example of alternative query in move_files cli

In [3]:
import datetime
from simex.utils.zendro import query_alternative_auxiliar_for_move_files_to_standard_directory, \
query_alternative_for_move_files_to_standard_directory

In [5]:
first_date_str = "2021-07-29"
second_date_str = "2021-08-26"
serial_number = "HLPXGM09048890"
file_type="image"
diff_dates = 29
query_result, operation_sgqlc = query_alternative_auxiliar_for_move_files_to_standard_directory(serial_number,
                                                                                                file_type)
device_deploymentsFilter_list = query_result["data"]["physical_devices"][0]["device_deploymentsFilter"]
format_string_data = "%Y-%m-%d"
list_dates_device_deployment = [d["date_deployment"].split('T')[0] for d in device_deploymentsFilter_list]
list_datetimes_device_deployment = [datetime.datetime.strptime(d["date_deployment"].split('T')[0],
                                                               format_string_data) for d in device_deploymentsFilter_list]
first_datetime  = datetime.datetime.strptime(first_date_str, format_string_data)
second_datetime = datetime.datetime.strptime(second_date_str, format_string_data)
diff_dates_datetime = datetime.timedelta(diff_dates)
list_datetimes_device_deployment_increased = [d + diff_dates_datetime/2 for d in list_datetimes_device_deployment]
k = 0
for datetimes_device_deployment in list_datetimes_device_deployment_increased:
    if first_datetime <= datetimes_device_deployment and datetimes_device_deployment <= second_datetime:
        idx_date = k
    k += 1
date_for_filter = device_deploymentsFilter_list[idx_date]["date_deployment"]

query_result, operation_sgqlc = query_alternative_for_move_files_to_standard_directory(serial_number,
                                                                                       date_for_filter,
                                                                                       file_type)
device_deploymentsFilter_list = query_result["data"]["physical_devices"][0]["device_deploymentsFilter"]

In [6]:
device_deploymentsFilter_list

[{'node': {'nomenclatura': '3_92_1_1338',
   'cat_integr': 'Integro',
   'ecosystems': {'name': 'Selvas humedas'}},
  'cumulus': {'name': '92'},
  'date_deployment': '2021-07-28T16:35:00.000Z',
  'latitude': 16.15832,
  'longitude': -90.90487}]

In [9]:
first_date_str = "2021-07-29"
second_date_str = "2021-08-28"
serial_number = "248D9B045EC9E6B1"
file_type="audio"
diff_dates = 30
query_result, operation_sgqlc = query_alternative_auxiliar_for_move_files_to_standard_directory(serial_number,
                                                                                                file_type)
device_deploymentsFilter_list = query_result["data"]["physical_devices"][0]["device_deploymentsFilter"]
format_string_data = "%Y-%m-%d"
list_dates_device_deployment = [d["date_deployment"].split('T')[0] for d in device_deploymentsFilter_list]
list_datetimes_device_deployment = [datetime.datetime.strptime(d["date_deployment"].split('T')[0],
                                                               format_string_data) for d in device_deploymentsFilter_list]
first_datetime  = datetime.datetime.strptime(first_date_str, format_string_data)
second_datetime = datetime.datetime.strptime(second_date_str, format_string_data)
diff_dates_datetime = datetime.timedelta(diff_dates)
list_datetimes_device_deployment_increased = [d + diff_dates_datetime/2 for d in list_datetimes_device_deployment]
k = 0
for datetimes_device_deployment in list_datetimes_device_deployment_increased:
    if first_datetime <= datetimes_device_deployment and datetimes_device_deployment <= second_datetime:
        idx_date = k
    k += 1
date_for_filter = device_deploymentsFilter_list[idx_date]["date_deployment"]

query_result, operation_sgqlc = query_alternative_for_move_files_to_standard_directory(serial_number,
                                                                                       date_for_filter,
                                                                                       file_type)
device_deploymentsFilter_list = query_result["data"]["physical_devices"][0]["device_deploymentsFilter"]

In [10]:
device_deploymentsFilter_list

[{'node': {'nomenclatura': '3_92_0_1339',
   'cat_integr': 'Degradado',
   'ecosystems': {'name': 'Selvas humedas'}},
  'cumulus': {'name': '92'},
  'date_deployment': '2021-07-29T17:05:00.000Z',
  'latitude': 16.1517,
  'longitude': -90.81243}]

# (Deprecated) Extract metadata and ingest it

In [12]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [13]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [14]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

## Check

In [15]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:34,559 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2021-12-15 11:35:34,561 [INFO]  Read metadata of JPG
2021-12-15 11:35:34,561 [INFO]  {}
2021-12-15 11:35:34,561 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:34,562 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/JPG


In [16]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:35,463 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI
2021-12-15 11:35:35,465 [INFO]  Read metadata of AVI
2021-12-15 11:35:35,465 [INFO]  {}
2021-12-15 11:35:35,465 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:35,465 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/AVI


In [17]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:35,910 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2021-12-15 11:35:35,912 [INFO]  Read metadata of WAV
2021-12-15 11:35:35,912 [INFO]  {}
2021-12-15 11:35:35,912 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:35,912 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/WAV
